In [1]:
import sys
sys.version

'3.12.9 | packaged by Anaconda, Inc. | (main, Feb  6 2025, 18:49:16) [MSC v.1929 64 bit (AMD64)]'

In [2]:
import os
from openai import OpenAI
import sys
sys.path.append('../..')
import utils
import openai



client=OpenAI(api_key=""

In [3]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content


In [4]:
def process_user_input(user_input,all_messages):
    delimiter = "```"
    def check_input_with_moderation(user_input):
        response = client.moderations.create(input=user_input)
        moderation_output = response.results[0]

        if moderation_output.flagged:
            print("Step 1: Input flagged by Moderation API.")
            return "Sorry, we cannot process this request."
        else:
            print("Step 1: Input passed moderation check.")
            return "Proceed to next step"
    check_input_with_moderation(user_input)
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    print("Step 2: Extracted list of products.")
    
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    
    product_information=utils.generate_output_string(category_and_product_list)
    print("Step 3: Looked up product information.")
    
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
     ]

    
    final_response = get_completion_from_messages(all_messages + messages)
    all_messages = all_messages + messages[1:]
    print("Step 4: Generated response to user question.")
    
    response = client.moderations.create(input=final_response)
    moderation_output = response.results[0]

    if moderation_output.flagged: 
        print("Step 5: Response flagged by Moderation API.")
        print("Sorry, we cannot provide this information.")

    else:
        print("Step 5: Response passed moderation check.")
        
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    say "yes" if so, otherwise "no"
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    print("Step 6: Model evaluated the response.")
    
    if evaluation_response=="yes":
        print("Step 7: Model approved the response.")
        return final_response,all_messages

    else:
        neg_string="I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        print("Step 7: Model disapproved the response.")
        return neg_string,all_messages

In [14]:
user_input= "tell me about FotoSnap Instant Camera"
all_messages=[]
response,context=process_user_input(user_input,all_messages)

Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 5: Response passed moderation check.
Step 6: Model evaluated the response.
Step 7: Model approved the response.


In [15]:
response

'The FotoSnap Instant Camera (model number FS-IC10) is a portable camera that allows you to create instant prints. It features a built-in flash, a selfie mirror for easy selfies, and is battery-powered for convenience. The camera is priced at $69.99 and comes with a 1-year warranty. Is there anything specific you would like to know about this camera or any other camera models?'

In [16]:
context

[{'role': 'user', 'content': '```tell me about FotoSnap Instant Camera```'},
 {'role': 'assistant',
  'content': 'Relevant product information:\n{\n    "name": "FotoSnap DSLR Camera",\n    "category": "Cameras and Camcorders",\n    "brand": "FotoSnap",\n    "model_number": "FS-DSLR200",\n    "warranty": "1 year",\n    "rating": 4.7,\n    "features": [\n        "24.2MP sensor",\n        "1080p video",\n        "3-inch LCD",\n        "Interchangeable lenses"\n    ],\n    "description": "Capture stunning photos and videos with this versatile DSLR camera.",\n    "price": 599.99\n}\n{\n    "name": "ActionCam 4K",\n    "category": "Cameras and Camcorders",\n    "brand": "ActionCam",\n    "model_number": "AC-4K",\n    "warranty": "1 year",\n    "rating": 4.4,\n    "features": [\n        "4K video",\n        "Waterproof",\n        "Image stabilization",\n        "Wi-Fi"\n    ],\n    "description": "Record your adventures with this rugged and compact 4K action camera.",\n    "price": 299.99\n}\

In [17]:
import panel as pn
pn.extension()
inp = pn.widgets.TextInput(placeholder='enter here')# inp is a text widget which creates the text box where the 
#user types the qtn
button_conversation = pn.widgets.Button(name="Ask Assistant")

In [18]:
def collect_messages(event=None, debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if inp.value == '':
        return
        inp.value = ''
        global context
    response,context=process_user_input(user_input,context)
    context.append({'role':'assistant', 'content':f"{response}"})# context already has user msg and product info hence we r append
# only the response to it to create a memory and to pass as the next context 

    panels.append(
    pn.Row('User:', pn.pane.Markdown(user_input, width=600))
)

    panels.append(
    pn.Row('Assistant:', pn.Column(
        pn.pane.Markdown(response, width=600),
        styles={'background': '#F6F6F6', 'padding': '10px', 'border-radius': '5px'}
    ))
)
    return pn.Column(*panels)

In [19]:
panels = []
context = [ {'role':'system', 'content':"You are Service Assistant"} ]  


In [20]:
interactive_conversation = pn.bind(collect_messages, button_conversation)# so eachtime the button is clicked the collect_messages 
#function runs


In [21]:
dashboard = pn.Column(
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
    pn.Row(inp),
    pn.Row(button_conversation)
)

dashboard

Column
    [0] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)
    [1] Row
        [0] TextInput(placeholder='enter here')
    [2] Row
        [0] Button(name='Ask Assistant')